# VencoPy Tutorial 1

This tutorial showcases the general structure and workflow of VencoPy, as well as some basic features of its 4 main classes:
- dataParsers
- tripDiaryBuilder
- gridModeler
- flexEstimator

## Setting up the working space

In [1]:
import os, sys
import pandas as pd
import numpy as np
import yaml
import pathlib

path = '../..'
os.chdir(path)

#sys.path.append("../../classes")
#sys.path.append("../../../vencopy")

from classes.dataParsers import DataParser
from classes.tripDiaryBuilders import TripDiaryBuilder
from classes.gridModelers import GridModeler
from classes.flexEstimators import FlexEstimator
from classes.evaluators import Evaluator

In [2]:
datasetID = 'MiD17'

pathGlobalConfig = pathlib.Path.cwd() / 'config' / 'globalConfig.yaml'
with open(pathGlobalConfig) as ipf:
    globalConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathLocalPathConfig = pathlib.Path.cwd()  / 'config' / 'localPathConfig.yaml'
with open(pathLocalPathConfig) as ipf:
    localPathConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathParseConfig = pathlib.Path.cwd()  / 'config' / 'parseConfig.yaml'
with open(pathParseConfig) as ipf:
    parseConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathTripConfig = pathlib.Path.cwd()  / 'config' / 'tripConfig.yaml'
with open(pathTripConfig) as ipf:
    tripConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathGridConfig = pathlib.Path.cwd()  / 'config' / 'gridConfig.yaml'
with open(pathGridConfig) as ipf:
    gridConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathEvaluatorConfig = pathlib.Path.cwd()  / 'config' / 'evaluatorConfig.yaml'
with open(pathEvaluatorConfig) as ipf:
    evaluatorConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathFlexConfig = pathlib.Path.cwd()  / 'config' / 'flexConfig.yaml'
with open(pathFlexConfig) as ipf:
    flexConfig = yaml.load(ipf, Loader=yaml.SafeLoader)

In [3]:
globalConfig

{'pathRelative': {'input': './inputData/',
  'plots': './output/plots/',
  'dataOutput': './output/data/',
  'scripts': './scripts/',
  'config': './config/'},
 'numberOfHours': 24,
 'files': {'inputDataScalars': 'VencoPy_scalarInput.xlsx',
  'MiD17': {'enryptedZipFileB1': 'B1_Standard-DatensatzpaketEncrypted.zip',
   'enryptedZipFileB2': 'B2_Regional-DatensatzpaketEncrypted.zip',
   'tripDataZipFileRaw': 'CSV/MiD2017_Regional_Wege.csv',
   'householdsDataRaw': 'MiD2017_Regional_Haushalte.csv',
   'personsDataRaw': 'MiD2017_Regional_Personen.csv',
   'tripsDataRaw': 'MiD2017_Regional_Wege.dta',
   'purposesProcessed': 'inputProfiles_Purpose',
   'inputDataDriveProfiles': 'inputProfiles_Drive',
   'inputDataPlugProfiles': 'inputProfiles_Plug',
   'vencoPyOutput': 'vencoPyOutput',
   'flexPlotName': 'flexProfileComparison'},
  'MiD08': {'householdsDataRaw': 'MiD2008_PUF_Haushalte.dta',
   'personDataRaw': 'MiD2008_PUF_Personen.dta',
   'tripsDataRaw': 'MiD2008_PUF_Wege.dta',
   'tripsPro

## _dataParser_ class

In [4]:
pathLocalPathConfig

WindowsPath('C:/8_Work/VencoPy/VencoPy_internal/vencopy/config/localPathConfig.yaml')

In [5]:
vpData = DataParser(datasetID=datasetID, parseConfig=parseConfig, globalConfig=globalConfig, localPathConfig=localPathConfig, loadEncrypted=False)

Parsing properties set up
Starting to retrieve local data file from C:\8_Work\Data\MiD17\B2_Regional-DatensatzpaketEncrypted\STATA\MiD2017_Regional_Wege.dta
Finished loading 960619 rows of raw data of type .dta
Finished harmonization of variables
Starting filtering, applying 8 filters.
The following values were taken into account after filtering:
{'isMIVDriver': 577155,
 'tripDistance': 892627,
 'tripEndClock': 960619,
 'tripEndHour': 896186,
 'tripIsIntermodal': 790421,
 'tripPurpose': 958025,
 'tripStartClock': 960619,
 'tripStartHour': 898923}
All filters combined yielded a total of 414683 was taken into account
This corresponds to 43.16831126596497 percent of the original data
Parsing completed


## _tripDiaryBuilder_ class

In [6]:
# Trip distance and purpose diary compositions
# FIX LOCAL PATHS IN GLOBAL CONFIG FOR XLSX FILES IN INPUTDATA FOLDER
vpTripDiary = TripDiaryBuilder(datasetID=datasetID, tripConfig=tripConfig, globalConfig=globalConfig, ParseData=vpData)

Calculating hourly shares
Trip distance diary setup starting
Starting trip purpose diary setup
Finished purpose replacements
There are 318 daily trip diaries.
Drive data and trip purposes written to files inputProfiles_Drive_sesRepro_MiD17.csv and inputProfiles_Purpose_sesRepro_MiD17.csv


In [7]:
# Evaluate drive and trip purpose profiles
#vpEval = Evaluator(globalConfig=globalConfig, evaluatorConfig=evaluatorConfig,
#                   parseData=pd.Series(data=vpData, index=[datasetID]), label='SESPaperTest')
#vpEval.hourlyAggregates = vpEval.calcVariableSpecAggregates(by=['tripStartWeekday'])
#vpEval.plotAggregates()

## _gridModeler_ class

In [8]:
# Grid model applications
vpGrid = GridModeler(gridConfig=gridConfig, globalConfig=globalConfig, datasetID=datasetID)
vpGrid.assignSimpleGridViaPurposes()
vpGrid.writeOutGridAvailability()

Starting with charge connection replacement of location purposes
Grid connection assignment complete


## _flexEstimator_ class

In [9]:
# Estimate charging flexibility based on driving profiles and charge connection
vpFlex = FlexEstimator(flexConfig=flexConfig, globalConfig=globalConfig, evaluatorConfig=evaluatorConfig, datasetID=datasetID, ParseData=vpData)
vpFlex.baseProfileCalculation()
vpFlex.filter()
vpFlex.aggregate()
vpFlex.correct()
vpFlex.normalize()
vpFlex.writeOut()

Reading Venco input scalars, drive profiles and boolean plug profiles
There are 318 drive profiles and 318 plug profiles.
Flex Estimator initialization complete
Starting with iteration 0
10753.398007982298
Starting with iteration 1
168.81332603686678
Starting with iteration 2
4.162500000000364
0.0
0.0
0.0
Base profile calculation complete for dataset MiD17
There are 310 considered profiles and 307 DSM eligible profiles.


In [10]:
#vpEval.plotProfiles(flexEstimator=vpFlex)